In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple

In [26]:
data_train = pd.read_csv("./data/train.csv")
data_test = pd.read_csv("./data/test.csv")



In [59]:
colomn_to_drop = pd.read_csv("./data/tabledrop.csv")
# ctd = list(set(list(colomn_to_drop["Colonne2"].values)))
ctd = ["FMONTH", "IMONTH", "IYEAR", "DISPCODE", "PVTRESD1", "RENTHOM1", "CERVSCRN", "CRVCLPAP", "USENOW3", "_MAM5023", "_HADCOLN", "_CRCREC2", "_RACEG22", "_LLCPWT"]
# ctd += list(set())

In [60]:



def clean_and_fill_data(data_train : pd.DataFrame, data_test : pd.DataFrame, column_to_drop) -> Tuple[pd.DataFrame, pd.DataFrame] :
    # ici on va drop les colonnes
    
    # Fin du dropage des colonnes
    
    indexs_to_fill_with_0 = []
    
    missing_values = data_train.isnull().sum()
    missing_values_treshold = missing_values[missing_values >= 1]
    
    for index in missing_values_treshold.index :
        possible_value = set(data_train[index].dropna())
        condition_categoriel = (len(possible_value) <= 100) # si on a moins de 100 réponse disponible on considère que c'est catégorielle. Donc on remplace par 0 au vue des questions.
        condition_categoriel = condition_categoriel or 999 in possible_value # Dans certaine question 999 c'est refused de répondre
        condition_categoriel = condition_categoriel or 777 in possible_value
        condition_categoriel = condition_categoriel or 999999 in possible_value
        condition_categoriel = condition_categoriel or 99 in possible_value
        
        if condition_categoriel :
            indexs_to_fill_with_0.append(index)

    missing_values = data_test.isnull().sum()
    missing_values_treshold = missing_values[missing_values >= 1]
    print("_RACE1" in missing_values_treshold.index)
    for index in missing_values_treshold.index :
        possible_value = set(data_test[index].dropna())
        condition_categoriel = (len(possible_value) <= 100) # si on a moins de 100 réponse disponible on considère que c'est catégorielle. Donc on remplace par 0 au vue des questions.
        condition_categoriel = condition_categoriel or 999 in possible_value # Dans certaine question 999 c'est refused de répondre
        condition_categoriel = condition_categoriel or 777 in possible_value
        condition_categoriel = condition_categoriel or 999999 in possible_value
        condition_categoriel = condition_categoriel or 99 in possible_value

        if condition_categoriel :
            indexs_to_fill_with_0.append(index)
        
    for index in indexs_to_fill_with_0 :
        data_train[index] = data_train[index].fillna(0)
        data_test[index] = data_test[index].fillna(0)
    
    
    indexs_to_fill_with_mean = ["_CLLCPWT", "WTKG3", "_BMI5"]
    for index in indexs_to_fill_with_mean :
        data_train.replace([np.inf, -np.inf], np.nan, inplace=True)
        data_test.replace([np.inf, -np.inf], np.nan, inplace=True)
        
        data_train[index] = data_train[index].fillna(data_train[index].dropna().mean())
        data_test[index] = data_test[index].fillna(data_test[index].dropna().mean())
    
    
    
    # On modifier la taille ici
    data_train = data_train.drop("HEIGHT3", axis=1)
    data_test = data_test.drop("HEIGHT3", axis=1)
    
    data_train["_PACKDAY"] = data_train["LCSNUMCG"]/20
    data_test["_PACKDAY"] = data_test["LCSNUMCG"]/20
    
    data_train["_PACKYRS"] = round(data_train["_YRSSMOK"]*data_train["_PACKDAY"])
    data_test["_PACKYRS"] = round(data_test["_YRSSMOK"]*data_test["_PACKDAY"])
    

    value_map = {
    1: 10,
    2: 5,
    3: 0,
    4: -5,
    5: -10,
    7: 0,
    9: 0,
    np.nan: 0  # Handle NaN explicitly
    }
    
    data_train["GENHLTH"] = data_train["GENHLTH"].replace(value_map)
    data_test["GENHLTH"] = data_test["GENHLTH"].replace(value_map) 

    value_map = {
        88 : 0,
        99: 0,
        77 : 15,
        np.nan: 0  # Handle NaN explicitly
    }
    data_train["PHYSHLTH"] = data_train["PHYSHLTH"].replace(value_map)
    data_test["PHYSHLTH"] = data_test["PHYSHLTH"].replace(value_map) 

    value_map = {
        88 : 0,
        99: 0,
        77 : 15,
        np.nan: 0  # Handle NaN explicitly
    }
    data_train["MENTHLTH"] = data_train["MENTHLTH"].replace(value_map)
    data_test["MENTHLTH"] = data_test["MENTHLTH"].replace(value_map)


    # data_train["_BMI5"] = (data_train["WTKG3"]/(data_train["HTM4"]*data_train["HTM4"])).apply(lambda x : max(1,x))
    # data_test["_BMI5"] = (data_test["WTKG3"]/(data_test["HTM4"]*data_test["HTM4"])).apply(lambda x : max(1,x))

    
    
    for column in column_to_drop :
        data_train = data_train.drop(column, axis=1)
        data_test = data_test.drop(column, axis=1)
        
    
    return data_train, data_test


    
data_train_clean, data_test_clean = clean_and_fill_data(data_train=data_train,
                                                        data_test=data_test,
                                                        column_to_drop=ctd)

# data_train_clean.to_csv("./data/train1.csv")
# data_test_clean.to_csv("./data/test1.csv")

print(np.isinf(data_train_clean).sum().sum())
print(np.isinf(data_test_clean).sum().sum())

print(data_train_clean.isnull().sum().sum())
print(data_test_clean.isnull().sum().sum())
data_test_clean.isnull().sum()[data_test_clean.isnull().sum() != 0]



False
0
0
0
0


Series([], dtype: int64)

In [63]:

data_train_clean = data_train_clean.drop("ID", axis=1)
data_train_clean.head()

,_STATE,IDATE,IDAY,SEQNO,_PSU,CTELENM1,COLGHOUS,STATERE1,CELPHON1,LADULT1,...,_LCSREC,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4,TARGET
0,2.0,5132022,13,2022000436,2.022000e+09,1.0,0.0,1.0,2.0,1.0,...,0.0,1.0,2.700000e+01,1.0,1.870000e+02,1.0,1.0,1.0,2.0,True
1,9.0,2282022,28,2022005381,2.022005e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,9.000000e+02,9.0,9.990000e+04,9.0,0.0,0.0,0.0,False
2,19.0,1202022,20,2022000927,2.022001e+09,1.0,0.0,1.0,2.0,1.0,...,0.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,0.0,0.0,9.0,False
3,39.0,8122022,12,2022014408,2.022014e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,0.0,0.0,2.0,False
4,25.0,7222022,22,2022010910,2.022011e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.700000e+01,2.0,4.670000e+02,1.0,0.0,0.0,2.0,False


In [64]:
from sklearn.ensemble import RandomForestClassifier

X = data_train_clean.iloc[:, :-1]  # All columns except the last one
y = data_train_clean.iloc[:, -1]   # The last column is the target
print(X.shape)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X, y)

(225000, 308)


RandomForestClassifier(random_state=42)

In [65]:
sample = data_test_clean.sample(n=75000, random_state=42)

In [66]:
sample.head()

,_STATE,IDATE,IDAY,SEQNO,_PSU,CTELENM1,COLGHOUS,STATERE1,CELPHON1,LADULT1,...,_LCSREC,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4,ID
26837,17.0,3102022,10,2022000793,2.022001e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,3.300000e+01,2.0,4.670000e+02,1.0,0.0,0.0,1.0,251837
2592,21.0,8102022,10,2022002875,2.022003e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,0.0,0.0,2.0,227592
18359,30.0,5052022,5,2022003476,2.022003e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,0.0,0.0,2.0,243359
73292,29.0,8292022,29,2022001227,2.022001e+09,1.0,0.0,1.0,2.0,1.0,...,0.0,1.0,3.300000e+01,1.0,4.670000e+02,1.0,1.0,1.0,9.0,298292
60127,4.0,4012022,1,2022002463,2.022002e+09,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,3.000000e+00,1.0,4.700000e+01,1.0,1.0,1.0,2.0,285127


In [69]:
id_pred = sample.iloc[:, -1]
sample = sample.iloc[:, :-1]
print(np.shape(sample))
y_pred = rf_clf.predict(sample)
print(y_pred)

(75000, 308)
[False False False ... False False False]


In [70]:
submit = pd.DataFrame(
    {
        "ID" : id_pred,
        "pred" : y_pred
    }
)

submit.to_csv("./data/submit.csv", index=False)